In [57]:
from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView
from ipyregulus import SidePanel
from ipyregulus.filters import * 

from ipywidgets import FloatSlider, ValueWidget

In [58]:
gauss = io.load('data/gauss4.p')
gauss.add_attr('linear', node_model(linear_model))
gauss.add_attr('fitness', fitness)
gauss.add_attr('relative_fitness', relative_fitness)
gauss.tree.add_attr('parent_fitness', parent_fitness)
gauss.tree.add_attr('child_fitness', child_fitness)
gauss.tree.add_attr('size', node_size)
gauss.tree.add_attr('rel_size', node_relative_size)
gauss.tree.add_attr('span', node_span)

In [59]:
from ipywidgets import Output
debug_view = Output(title='debug')
debug_view

Output()

In [60]:
def update_view(view, f):
    def _inner():
        view.show = tree_filter(view.tree, f)
    return _inner 

#### Attribue Filter

In [61]:
v3 = TreeView(gauss.tree, measure='span')

In [62]:
f3 = AttrFilter(attr='span', func=lambda x, v: v <=x)

In [63]:
m3 = Monitor(f3, func=update_view(v3, f3))

In [64]:
p3 = SidePanel(title='panel 3')
with p3:
    display(v3, f3)

In [65]:
f3.attr = 'span'

In [10]:
v3.measure = 'child_fitness'

In [12]:
f3.func = lambda x,v: x <=v

#### Basic View Filter

In [16]:
v1 = TreeView(gauss.tree, measure='span')
v1

TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.5392374574080602, 1: 0.00980789566443424, 2:…

In [17]:
f1 = UIFilter(func=lambda v, tree, node: tree.attr['fitness'][node]<v, 
             ui=FloatSlider(min=0, max=1, step=0.01, value=0.5))
f1

FloatSlider(value=0.5, max=1.0, step=0.01)

In [18]:
m1 = Monitor(monitor=f1, func=update_view(v1, f1))

#### Reduce filter

In [19]:
reduce_func = lambda v, tree, node: tree.attr['span'][node]>v

In [20]:
r2 = UIFilter(func=reduce_func, 
             ui=FloatSlider(min=0, max=1, step=0.01, value=0.5))

In [21]:
def reduce_view(view, tree, f):
    def _inner():
        view.ref = reduce(tree, filter=f)
    return _inner

In [22]:
v2 = TreeView(gauss.tree, measure='fitness')
v2

TreeView(field='fitness', tree_model=TreeWidget(attrs={'fitness': {0: 0.3744578217446122, 1: 0.385549803155957…

In [23]:
m2 = Monitor(monitor=r2, func=reduce_view(v2, gauss.tree, r2))

In [24]:
r2

FloatSlider(value=0.5, max=1.0, step=0.01)

In [25]:
v2.measure='child_fitness'

In [27]:
p2 = SidePanel(title='Reduced Tree')
with p2:
    display(v2, r2)

In [21]:
f2 = UIFilter(func=lambda v, tree, node: tree.attr['span'][node]>v, 
             ui=FloatSlider(min=0, max=1, step=0.01, value=0.5))

In [22]:
m2 = Monitor(monitor=f2, func=update_view(v2, f2))

In [23]:
f2

FloatSlider(value=0.5, max=1.0, step=0.01)

In [39]:
p1 = SidePanel(title='Valerio')

In [40]:
with p1:
    display(v1, f)

In [40]:
def grand_parent(context, node):
    if node.id == -1 or node.parent.id == -1 or node.parent.parent.id == -1:
        return 0
    rel = context['relative_fitness']
    return abs(rel[node, node.parent.parent] - rel[node, node.parent])

In [41]:
v3.tree.add_attr('grandparent', grand_parent)

In [42]:
v3.measure = 'grandparent'

In [43]:
f3.attr = 'grandparent'

In [44]:
f3.func = lambda x, v: v <=x

In [46]:
v3.measure = 'fitness'

In [47]:
f3.attr = 'fitness'